In [273]:
import os
import pandas as pd
import numpy as np
import tabula
from tabula.io import read_pdf

# read all pages of pdf document
file_name = "Hempstead.pdf"
dfs = tabula.io.read_pdf(file_name, pages=f'1-10', lattice=True)

In [535]:
import tabula

all_dfs = pd.DataFrame()
for x in range(12):
    dfs = tabula.read_pdf("Hempstead.pdf",pages=f'{x*800+1}-{(x+1)*800}', lattice=True)
    df = pd.concat(dfs)
    all_dfs = pd.concat([all_dfs, df])
last_df = tabula.read_pdf("Hempstead.pdf",pages='9601-10120', lattice=True)
last_df = pd.concat(last_df)
all_dfs = pd.concat([all_dfs, last_df])

all_dfs.reset_index(inplace=True)

In [537]:
test_dfs = all_dfs.copy()
# Find rows with null values in the third column
null_rows = all_dfs[all_dfs["Last Known Owner Name\rLast Known Mailing Address"].isnull()].index.to_list()

# Slice the DataFrame to keep the first two columns and columns 4-10
new_columns = ['index'] + list(all_dfs.columns[2:])

# Shift the values of columns 4-10 to become columns 3-9
all_dfs.loc[null_rows, new_columns] = all_dfs.loc[null_rows, new_columns].shift(-1, axis=1)

# drop two columns
all_dfs = all_dfs.drop(["index","Unnamed: 0"], axis=1)
# Display the updated DataFrame
all_dfs.iloc[808:811]

KeyError: "['index'] not in index"

In [543]:
# concatenate all tables in pdf documents as one dataframe
#all_dfs=pd.concat(dfs)

# take first 13 columns (some tables had a ghost 14th column)
#all_dfs=all_dfs.iloc[:,:13]

# rename the dataframe's columns (names were long and poorly formatted)
all_dfs.columns=["Property Info",
"Name and Address",
"Codes",
"Full Market Value",
"Land Assessed Value",
"Total Assessed Value",
"Exempt Code",
"Exemption Amount",
"Village Codes",
"Rate Codes",
"Tax District Percent",
"Total Taxable Value Town",
"Total Taxable Value County"]

# create one dataframe from the columns that do not need to be expanded (to be joined at the end)
all_other_columns_df=test_dfs[["Full Market Value",
"Land Assessed Value",
"Total Assessed Value",
"Exempt Code",
"Exemption Amount",
"Village Codes",
"Rate Codes",
"Tax District Percent",
"Total Taxable Value Town",
"Total Taxable Value County"]]

## Splitting First column into separate columns

In [545]:
# create multiple columns from the first column's rows (they are read and split on "\r" character)
series=all_dfs["Property Info"].apply(lambda x:x.split("\r"))
series_df=pd.DataFrame(data=series)
series_df.columns=["First_Column"]
series_df["length"]=series_df["First_Column"].apply(lambda x:len(x))
all_series=series_df["First_Column"].tolist()

# since many cells do not have a value for Lot Grouping, insert a blank value for that place in the list (so the columns aren't distorted)
for series in all_series:
    if 'acres' in series[2]:
        series.insert(1,"")

# take first 5 items from the list (which will create 5 columns in final dataframe). The bottom of the first cell has long code we don't need.
all_series=[series[:5] for series in all_series]

# create a dtaframe of 5 columns from the original first column
first_column_df=pd.DataFrame(all_series, columns=['Section-Block-Lot', 'Lot Grouping', 'Address','Lot Size','Liber'])

## Splitting Second column into separate columns

In [547]:
# split on "\r" character and handle issue with float that cannot be split (this came up in one of the 10,000 pages)
address_series=all_dfs['Name and Address'].apply(lambda x:x.split("\r") if not(isinstance(x, float)) else ["","","",""])
address_series_df=pd.DataFrame(data=address_series)
address_series_df.columns=["Second_Column"]
address_series_df["length"]=address_series_df["Second_Column"].apply(lambda x:len(x))
all_address_series=address_series_df["Second_Column"].tolist()

address_series_df["last_value_last_row"] = address_series_df["Second_Column"].apply(lambda x: x[-1].split(" ")[-1])
address_series_df["last_value_second_last_row"] = address_series_df["Second_Column"].apply(lambda x: x[-2].split(" ")[-1])
address_series_df["last_value_third_last_row"] = address_series_df["Second_Column"].apply(lambda x: x[-3].split(" ")[-1] if len(x)>2 else "hello")



# Define the condition using str.match()
# Apply the condition and select values from different columns
address_series_df["zip_code_value"] = np.where(address_series_df["last_value_last_row"].str.match(r'\d{5}+'), \
                                             address_series_df["last_value_last_row"],\
                                      np.where(address_series_df["last_value_second_last_row"].str.match(r'\d{5}+'),\
                                             address_series_df["last_value_second_last_row"],\
                                              address_series_df["last_value_third_last_row"]))

(address_series_df["zip_code_row"],address_series_df["address_code_row"]) = \
                    np.where(address_series_df["last_value_last_row"].str.match(r'\d{5}+'), \
                    (address_series_df["Second_Column"].str.get(-1),address_series_df["Second_Column"].str.get(-2)),\
                     np.where(address_series_df["last_value_second_last_row"].str.match(r'\d{5}+'),\
                    (address_series_df["Second_Column"].str.get(-2),address_series_df["Second_Column"].str.get(-3)),\
                    (address_series_df["Second_Column"].str.get(-3),address_series_df["Second_Column"].str.get(-4))))

address_series_df["name"] = address_series_df["Second_Column"].str.get(0)


In [627]:
# split on "\r" character 
codes_series=all_dfs['Codes'].apply(lambda x:x.split("\r"))
codes_series_df=pd.DataFrame(data=codes_series)
codes_series_df.columns=["Third_Column"]
all_codes_series=codes_series_df["Third_Column"].tolist()

# create a dtaframe of 6 columns from the original third column
third_column_df=pd.DataFrame(all_codes_series, columns=['Roll Section', 'SWIS Code',"Sch SWIS Code",'School Code','PUC - Class',"Percent Value"])



In [641]:
def assessed_value(x):
    if type(x) == int:
        return x
    elif type(x) == float:
        return int(x)
    else:
        return x.split("\r")[-1]
    
all_dfs["Total Assessed Value"] = all_dfs["Total Assessed Value"].apply(lambda x: assessed_value(x))

pd.concat([address_series_df[['name', 'address_code_row', 'zip_code_row']],\
           all_dfs['Total Assessed Value'],third_column_df],axis=1)\
           .to_csv("final_df.csv",index=False)


In [ ]:
def assessed_value(x):
    if type(x) == int:
        return x
    elif type(x) == float:
        return int(x)
    else:
        return x.split("\r")[-1]
    
all_dfs["Total Assessed Value"] = all_dfs["Total Assessed Value"].apply(lambda x: assessed_value(x))

In [185]:
# merge first, second, third dataframes with the rest of the columns
final_df=pd.concat([first_column_df,second_column_df,third_column_df,all_other_columns_df.reset_index()],axis=1)

In [ ]:
#write final dataframe to csv
final_df.to_csv("All_Properties.csv", index=False)

In [568]:
pd.DataFrame(all_dfs["Total Assessed Value"]).to_csv("assessed.csv", index=False)#.apply(lambda y: get_assessed_value(y)).value_counts()